In [ ]:

import pandas as pd
import numpy as np 
import re 
import nltk
nltk.download("punkt")
from nltk.corpus import stopwords 
sw=stopwords.words('english') 
from nltk.util import ngrams
from lib.config import connstr
from lib.utils import * 
from sklearn.feature_extraction.text import CountVectorizer
import warnings
warnings.filterwarnings('ignore') 
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.svm import LinearSVR, SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.model_selection import train_test_split 
from sklearn.preprocessing import StandardScaler 
from sklearn.linear_model import LogisticRegression 


In [ ]:
USE_CACHE=False
def extract_ngrams(data, num):
    n_grams = ngrams(nltk.word_tokenize(data), num)
    return [ ' '.join(grams) for grams in n_grams]

In [188]:
if not USE_CACHE:
    df=sql_todf(f"""
        SELECT FK_QuestionType, QuestionText
        FROM fsma_Questions WHERE FK_QuestionType IS NOT NULL AND QuestionText IS NOT NULL 
        AND FK_QuestionType IN (1,2 )
        """, connstr)   
    df['NQuestionText']=df['QuestionText'].apply(lambda s: re.sub('[^a-z\s\-]','',s.lower() )) 
    df['NQuestionText'].dropna(inplace=True)
    df['NQuestionText']=df['NQuestionText'].apply(lambda s: ' '.join([s for s in s.split() if s.lower() not in sw][:6]).upper()) 

    vectorizer = CountVectorizer()
    X = vectorizer.fit_transform(df['NQuestionText'])  
     
    dff = pd.DataFrame(columns=vectorizer.get_feature_names(), data=X.toarray()) 
    m=pd.merge(df, dff, left_index=True, right_index=True)
    m.to_csv(r'c:\temp\cache.csv', index=False)
df=pd.read_csv(r'c:\temp\cache.csv')

In [192]:
dff=df.copy()
dff.loc[220:285,['FK_QuestionType','NQuestionText']]

,FK_QuestionType,NQuestionText
220,2,PROVIDE NUMBER TYPES OPERATING SYSTEM SOFTWARE
221,2,PROVIDE NUMBER OPERATING SYSTEM SOFTWARE USE
222,2,PROVIDE NUMBER ENTERPRISE-WIDE APPLICATIONS EG...
223,2,PROVIDE NUMBER ENTERPRISE-WIDE APPLICATIONS ST...
224,2,PROVIDE NUMBER AGENCY INFORMATION TECHNOLOGY A...
...,...,...
281,1,AGENCY POLICY PLACE ENSURE PERSONNEL EMPLOYEES
282,1,AGENCY PROGRAM JOB-SPECIFIC COMPREHENSIVE INFO...
283,1,DETERMINING WHETHER PIA NEEDED
284,1,CONDUCTING PIA


In [193]:
df.drop(['NQuestionText', 'QuestionText'], inplace=True, axis=1)
y = df['FK_QuestionType'].values 
X = df.drop(['FK_QuestionType' ], axis=1).values
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

model = LogisticRegression()
model.fit(X_train, y_train)
model.score(X_test, y_test)

0.9767025089605734

In [194]:
def predict(s):
    arr=np.zeros(len(vectorizer.get_feature_names()), dtype=int)
    lst=s.split(' ')
    for w in lst:
        if w in vectorizer.get_feature_names():
            pos=vectorizer.get_feature_names().index(w)
            arr[pos]=1 
    return arr 

In [196]:
arr=predict('AGENCY POLICY PLACE ENSURE PERSONNEL EMPLOYEES') 
prediction =model.predict([arr])
prediction[0]

2